# Simulation Study: Open System (Bootstrap)

In [1]:
### LIBRARY ###
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random as rand

from mm_temptations_functions import *
from scipy.optimize import minimize, Bounds, LinearConstraint

from multiprocess import Pool
from functools import partial

import pickle

In [2]:
planting(456)

In [3]:
# set system wide param
k = 7 # no. of states
timeCourse = 16 # total timepoints
u = [1] * int(timeCourse) # time interval
n_param = int(k/2 * (k-1)) # number of parameters

sampRange_ub = 4
xterm = 1e-15
max_iter = 5e3
numCores = 5
mc_iter = 5

In [4]:
file = "simResults/simulated_CN_Q_true.pkl"
with open(file, 'rb') as f:
    Q_true = pickle.load(f)

In [5]:
filename = "simDatasets/simulated_Open_propDicts_groupSize5.pkl"
with open(filename, 'rb') as f:
    [pi_hat_donors, T_donors, u_donors] = pickle.load(f)

In [6]:
filename = "simResults/simulated_Open_estimates_50reps.pkl"
with open(filename, 'rb') as f:
                [estimated_theta, estimated_cost] = pickle.load(f)

In [7]:
point_estimate = estimated_theta[0,:]

### Bootstrap Uncertainty

In [8]:
# set variables
Q_template = make_theta0(k, 'Q')

reg_bounds = def_bounds(n_param, k, account_ingress=False)
reg_con = def_constraints(n_param, k)

In [9]:
B = 1500
startIteration = 1057
optimiserArgs = {'costFunc': calc_cost_donors, 'args': (pi_hat_donors, T_donors, k, u_donors, Q_template), 'bounds': reg_bounds, 'constraints': reg_con, 'sampRange_ub': sampRange_ub}
options = {'hessian': None, 'xterm': xterm, 'max_iter': max_iter}

mcArgs = {'mciter': mc_iter, 'n_cores': numCores}

In [10]:
# load old check point into new checkpoint
theta_boot = np.zeros((B,n_param))
checkpoint_filename = "simResults/simOpen_BScheckpoint_theta_boot.pkl"
with open(checkpoint_filename, 'rb') as f:
    theta_boot = pickle.load(f)
    
with open(checkpoint_filename, 'wb') as f:
    pickle.dump(theta_boot,f)

In [11]:
# create new checkpoint array
#theta_boot = np.zeros((B,n_param))
#checkpoint_filename = "simResults/simOpen_BScheckpoint_theta_boot.pkl"   
#with open(checkpoint_filename, 'wb') as f:
#    pickle.dump(theta_boot,f)

In [12]:
residResamp=bootstrap_resResamp_perDonor(point_estimate=point_estimate, n_bootstrapSamples=B, n_param=n_param, theta_generator=make_theta0, mcArgs=mcArgs, optimiserArgs=optimiserArgs, options=options, checkpoint_filename=checkpoint_filename, startIteration=startIteration)

 28% (125 of 443) |#    | Elapsed Time: 5 days, 4:01:57 ETA:  10 days, 20:41:27
 28% (126 of 443) |#      | Elapsed Time: 5 days, 4:44:29 ETA:  9 days, 8:41:59
 28% (127 of 443) |#    | Elapsed Time: 5 days, 5:43:27 ETA:  12 days, 22:35:57
 28% (128 of 443) |#     | Elapsed Time: 5 days, 6:30:15 ETA:  10 days, 5:42:07
 29% (129 of 443) |#    | Elapsed Time: 5 days, 7:51:50 ETA:  17 days, 18:55:42
 29% (130 of 443) |#     | Elapsed Time: 5 days, 9:06:54 ETA:  16 days, 7:36:57
 29% (131 of 443) |#   | Elapsed Time: 5 days, 10:28:23 ETA:  17 days, 15:40:19
 29% (132 of 443) |#   | Elapsed Time: 5 days, 11:31:05 ETA:  13 days, 12:58:53
 30% (133 of 443) |#    | Elapsed Time: 5 days, 13:04:05 ETA:  20 days, 0:31:56
 30% (134 of 443) |#     | Elapsed Time: 5 days, 13:47:09 ETA:  9 days, 5:44:45
 30% (135 of 443) |#     | Elapsed Time: 5 days, 14:30:45 ETA:  9 days, 7:52:18
 30% (136 of 443) |#    | Elapsed Time: 5 days, 15:28:47 ETA:  12 days, 8:55:39
 30% (137 of 443) |#    | Elapsed Time: 

Compiling results...


In [13]:
filename = "simResults/simOpen_BS_allTime_allDonor_5state.pkl"
with open(filename, 'wb') as f:
    pickle.dump(residResamp, f)